In [253]:
import pprint
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.lancaster import LancasterStemmer
import ast
import numpy as np
import os
import ast
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os.path
from datetime import datetime
from collections import Counter

nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nefarion/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/nefarion/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/nefarion/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [254]:
os.chdir("/mnt/f/Linda/Work work/Categorization")

input_file_relative = "Datasets/Feature_dataset.csv"
output_file_relative = "Datasets/hopefully_the_end.csv"
month = "October"
char_blacklist = list(chr(i) for i in range(32, 127) if i <= 64 or i >= 91 and i <= 96 or i >= 123)
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(char_blacklist)
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
words_threshold = 10
top = 2500
toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
words_frequency = {}

In [255]:
# Read new generated data set file
df = pd.read_csv(input_file_relative)

# Take only English documents - above a certain threshold

In [256]:
english_confidence = []
english_tolerance = 20
for i, row in df.iterrows():
    english_words = 0
    words = ast.literal_eval(row['tokens_en'])
    for word in words:
        if word.lower() in english_vocab:
            english_words += 1
    english_confidence.append(english_words / len(words) * 100)
df['english:confidence'] = english_confidence
df = df[df['english:confidence'] > english_tolerance]

# Make the most popular word list for each category

In [257]:
top = 10
words_frequency = {}
stemmer = LancasterStemmer()
for category in set(df['main_category'].values):
    all_words = []
    for row in df[df['main_category'] == category]['tokens_en'].tolist():
        for word in ast.literal_eval(row):
            all_words.append(word)
    allWordExceptStopDist = nltk.FreqDist(
        stemmer.stem(w.lower()) for w in all_words if w not in stopwords and len(w) >= 3 and w[0] not in char_blacklist
    )

    most_common = allWordExceptStopDist.most_common(top)
    words_frequency[category] = [word for word, number in most_common]

In [258]:
words_frequency

{'Web_Catalogue': ['us',
  'market',
  'on',
  'usernam',
  'password',
  'dream',
  'anonym',
  'reg',
  'phish',
  'mir'],
 'Gift_Cards': ['card',
  'pric',
  'pay',
  'contact',
  'amazon',
  'bitcoin',
  'market',
  'gift',
  'payp',
  'giftcard'],
 'Porn': ['sit',
  'new',
  "n't",
  'sex',
  'adult',
  'mat',
  'view',
  'upload',
  'porn',
  'account'],
 'Guns': ['gun',
  'euro',
  'numb',
  'on',
  'deal',
  'onionland',
  'buy',
  'ammo',
  'bitcoin',
  'logo.png'],
 'Computer_and_Electronics': ['us',
  'host',
  'serv',
  'man',
  'report',
  'docu',
  'op',
  'direct',
  'pack',
  'pag'],
 'Crypto_Currency': ['us',
  'mix',
  'blend',
  'bitcoin',
  'transact',
  'address',
  'hour',
  'serv',
  'on',
  'inst'],
 'Skimming': ['read',
  'skim',
  'receiv',
  'card',
  'track',
  'atm',
  'pric',
  'produc',
  'work',
  'ord'],
 'Internet_and_Telecom': ['serv',
  'publ',
  'off',
  'us',
  'fre',
  'mail',
  'gen',
  'secmail',
  'research',
  'am'],
 'Shopping': ['hom',
  'lo

In [259]:
# from collections import Counter
# 
# features = np.zeros(df.shape[0] * top).reshape(df.shape[0], top)
# labels = np.zeros(df.shape[0])
# counter = 0
# categories = list(set(df['main_category'].values)) 
# for i, row in df.iterrows():
#     c = [word for word, word_count in Counter(ast.literal_eval(row['tokens_en'])).most_common(top)]
#     category = row['main_category']
#     labels[counter] = categories.index(category)
#     for word in c:
#         if word in words_frequency[row['main_category']]:
#             features[counter][words_frequency[category].index(word)] = 1
#     counter += 1
# 
# 

In [260]:
# from sklearn.metrics import accuracy_score
# from scipy.sparse import coo_matrix
# X_sparse = coo_matrix(features)
# 
# from sklearn.utils import shuffle
# X, X_sparse, y = shuffle(features, X_sparse, labels, random_state=0)
# 
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# 
# from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression()
# lr.fit(X_train, y_train)
# predictions = lr.predict(X_test)
# score = lr.score(X_test, y_test)
# print('LogisticRegression')
# print('Score: ', score)
# print('Top: ', top)
# print('Dataset length: ', df.shape[0])
# print()
# 
# from sklearn.svm import LinearSVC
# clf = LinearSVC()
# clf.fit(X_train, y_train)
# predictions = clf.predict(X_test)
# score = clf.score(X_test, y_test)
# print('SVM')
# print('Score: ', score)
# print('Top: ', top)
# print('Dataset length: ', df.shape[0])
# 

In [261]:
# import pandas as pd
# import nltk
# import ast
# import numpy as np
# import os
# import ast
# import urllib.request
# from urllib.request import urlopen
# from bs4 import BeautifulSoup
# import os.path
# nltk.download('stopwords')
# nltk.download('words')
# nltk.download('punkt')

# Dataset creation if it is not existing.
__Dataset is filtered by these set of rules:__
1. Main category != Not_working (Exclude non working URL's)
2. Main category:confidence > 0.5 (Leave url's with likely know categories)
3. Non responding URL's are excluded
4. Non english language URL's are excluded.

### Caution, the full data set creation may take ~15 hours.

In [262]:
# def no_filter_data():
#     file = input_file_relative
#     df = pd.read_csv(file)[['main_category', 'main_category:confidence', 'url']]
#     df = df[(df['main_category'] != 'Not_working') & (df['main_category:confidence'] > 0.5)]
#     df['tokens_en'] = ''
#     
#     counter = 0
#     for i, row in df.iterrows():
#         counter += 1
#         url = row['url']
#         print("{}, {}/{}".format(url, counter, len(df)))
# 
#         try:
#             hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
#                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
#                'Accept-Encoding': 'none',
#                'Accept-Language': 'en-US,en;q=0.8',
#                'Connection': 'keep-alive'}
#             req = urllib.request.Request(url, headers=hdr)
#             html = urlopen(req).read()
# #             html = urlopen('http://' + row['url'], timeout=15).read()
#         except Exception as inst:
#             print(f" {url} FAILED. because of {inst}")
#             continue
# 
#         soup = BeautifulSoup(html, "html.parser")
#         [tag.decompose() for tag in soup("script")]
#         [tag.decompose() for tag in soup("style")]
#         text = soup.get_text()
#         lines = (line.strip() for line in text.splitlines())
#         chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
#         text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
#         tokens = nltk.word_tokenize(text)
# 
#         df.at[i, 'tokens_en'] = tokens if len(tokens) > 0 else ''
#         
#     df = df[~df['tokens_en'].isnull()]
#     df.to_csv(output_file_relative)
#     
# no_filter_data()

### Remove most frequent words in all categories
### Get all top words of all categories combined
### Create a keyword processor for each category

In [263]:
from flashtext.keyword import KeywordProcessor
from collections import Counter
words = []
all_keywords = []
word_processors = {}
for category in words_frequency.keys():
    words.extend(words_frequency[category][0:15])
words_counter = Counter(words)
words_filter = {x : words_counter[x] for x in words_counter if words_counter[x] >= 7}
words_stop = list(words_filter.keys())
for category in words_frequency.keys():
    words_frequency[category] = [word for word in words_frequency[category] if word not in words_stop]
    all_keywords = all_keywords + words_frequency[category]
    word_processor = KeywordProcessor()
    for word in words_frequency[category]:
        word_processor.add_keyword(word)
    word_processors[category] = word_processor
    
all_words_processor = KeywordProcessor()
for word in all_keywords:
    all_words_processor.add_keyword(word) 

In [264]:
words_filter

{}

### Create a percentage function 

In [265]:
# def compute_percentage(dum0, dumx):
#     try:
#         ans=float(dumx)/float(dum0)
#         ans=ans*100
#     except:
#         return 0
#     else:
#         return ans
#    

### Create a function to find the most probable category 

In [266]:
# def guess_category(text, index):
#     x=str(text)
#     total_matches = len(all_words_processor.extract_keywords(x))
#     if total_matches == 0:
#         return 'Not working'
#     
#     matched_keywords_count = {}
#     for p_key in word_processors:
#         processor = word_processors[p_key]
#         matched_keywords_count[p_key] = len(processor.extract_keywords(x))
#         
#     match_per_category = {}
#     for tk_key in matched_keywords_count:
#         matched = matched_keywords_count[tk_key]
#         match_per_category[tk_key] = float(compute_percentage(total_matches, matched))
#         
#     max_prob_category = max(match_per_category, key=(lambda key: match_per_category[key]))
#     return max_prob_category 

# For each website use only its tokenized words and its category

In [267]:
words = []
classes = []
documents = []
counter = 0
for i, row in df.iterrows():
    c = [word for word, word_count in Counter(ast.literal_eval(row['tokens_en'])).most_common(top)]
    documents.append((c, row['main_category']))
    
words = all_keywords
classes = list(words_frequency)    

# Convert the Language Words into mathematical notations

In [270]:
training = []
output = [] 
output_empty = [0] * len(classes)
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    training.append(bag)
    # output is a '0' for each tag and '1' for current category
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)
    
pprint.pprint(output)
pprint.pprint(training)

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

# Create features and labels for Machine learning training

In [269]:
from collections import Counter

features = np.zeros(df.shape[0] * top).reshape(df.shape[0], top)
labels = np.zeros(df.shape[0])
counter = 0
for i, row in df.iterrows():
    c = [word for word, word_count in Counter(ast.literal_eval(row['tokens_en'])).most_common(top)]
    labels[counter] = list(set(df['main_category'].values)).index(row['main_category'])
    for word in c:
        if word in words_frequency[row['main_category']]:
            features[counter][words_frequency[row['main_category']].index(word)] = 1
    counter += 1

# Create seperate training/testing datasets and shuffle them

In [46]:
from sklearn.metrics import accuracy_score
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(features)

from sklearn.utils import shuffle
X, X_sparse, y = shuffle(features, X_sparse, labels, random_state=0)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print('features____________________________________________:')
pprint.pprint(features)
print('labels____________________________________________:')
pprint.pprint(labels)

features____________________________________________:
array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])
labels____________________________________________:
array([ 6., 11., 11.,  5.,  0.,  5.,  5.,  9., 10.,  4.,  6.,  4., 11.,
       11.,  6.,  4., 11.,  2., 11.,  9., 11.,  5.,  3., 10.,  2.,  8.,
        2.,  8.,  5.,  5.,  2., 10.,  6.,  0.,  0.,  1.,  0.,  5., 12.,
        7., 11.])


# Predictions

In [47]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)
score = lr.score(X_test, y_test)
print('LogisticRegression')
print('Score: ', score)
print('Top: ', top)
print('Tolerance: ', english_tolerance)
print('Dataset length: ', df.shape[0])
print()
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
predictions = dtc.predict(X_test)
score = dtc.score(X_test, y_test)
print('DecisionTreeClassifier')
print('Score: ', score)
print('Top: ', top)
print('Tolerance: ', english_tolerance)
print('Dataset length: ', df.shape[0])
print()
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train, y_train) 
predictions = clf.predict(X_test)
score = clf.score(X_test, y_test)
print('SVM')
print('Score: ', score)
print('Top: ', top)
print('Tolerance: ', english_tolerance)
print('Dataset length: ', df.shape[0])

LogisticRegression
Score:  0.14285714285714285
Top:  2500
Tolerance:  20
Dataset length:  41

DecisionTreeClassifier
Score:  0.21428571428571427
Top:  2500
Tolerance:  20
Dataset length:  41

SVM
Score:  0.21428571428571427
Top:  2500
Tolerance:  20
Dataset length:  41


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


### Save ML model

In [48]:
month = 'October'
from sklearn.externals import joblib
filename = "Models/{}/LR_model_{}.joblib".format(month, month)
joblib.dump(lr, filename) 

import pickle
words_filename = "Models/{}/word_frequency_{}.picle".format(month, month)
pickle_out = open(words_filename,"wb")
pickle.dump(words_frequency, pickle_out)
pickle_out.close()
    
filename = "Models/{}/LR_maxtrain_{}.joblib".format(month, month)
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X, y)
joblib.dump(lr, filename) 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


['Models/October/LR_maxtrain_October.joblib']

In [49]:
# import matplotlib.pyplot as plt; plt.rcdefaults()
# import numpy as np
# import matplotlib.pyplot as plt
 
# objects = ('English', 'Italic', 'Russian', 'Japan', 'China', 'Belgium')
# y_pos = np.arange(len(objects))
# performance = [8143,260,646,338,125,100]
 
# plt.bar(y_pos, performance, align='center', alpha=0.5)
# plt.xticks(y_pos, objects)
# plt.ylabel('URLs')
# plt.title('Languages diversity in the data set')
 
# plt.show()
# plt.savefig("language_diversity.png")
# df[df['main_category'] == 'Business_and_Industry']['url']

In [50]:
# import matplotlib.pyplot as plt; plt.rcdefaults()
# import numpy as np
# import matplotlib.pyplot as plt
# from collections import Counter

# words = []
# for category in words_frequency.keys():
#     words.extend(words_frequency[category][0:15])
# words_counter = Counter(words)
# words_filter = {x : words_counter[x] for x in words_counter if words_counter[x] >= 7}
# objects = tuple(words_filter.keys())
# y_pos = np.arange(len(objects))
# performance = list(words_filter.values())

# plt.barh(y_pos, performance, align='center', alpha=1)
# plt.xticks(range(1, max(performance) + 1))
# plt.yticks(y_pos, objects)
# plt.xlabel('Word diversity in categories (TOP 15 words)')
# plt.title('Words diversity in each category TOP 15 most frequent words')
 
# plt.show()
# plt.savefig("words_diversity.png")